In [1]:
# Familiar imports
import numpy as np
import pandas as pd
import random
import os
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error

import lightgbm as lgb
#import xgboost as xgb
#import catboost as ctb

import warnings
warnings.simplefilter('ignore')

# Parameters

In [2]:
target = 'loss'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 17
    CVSEED = 17
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 10
    N_ESTIMATORS = 10000
    EARLY_STOPPING_ROUNDS = 200
    VERBOSE = 1000
    SEED = 17
    CVSEED = 17
    #N_ITERS = 10

In [3]:
def set_seed(seed=17):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

# Load data

In [4]:
INPUT = Path("../input/tabular-playground-series-aug-2021")

train = pd.read_csv(INPUT / "train.csv")
test = pd.read_csv(INPUT / "test.csv")
submission = pd.read_csv(INPUT / "sample_submission.csv")

# Preprocessing

In [5]:
scale_features = [col for col in test.columns if 'f' in col]

ss = StandardScaler()
train[scale_features] = ss.fit_transform(train[scale_features])
test[scale_features] = ss.transform(test[scale_features])

In [6]:
# Swap noise

# Random
def apply_noise_rn(df, p=.75):
    should_not_swap = np.random.binomial(1, p, df.shape)
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Row-wise
def apply_noise_row(df, p=.75):
    should_not_swap = np.zeros(df.shape)
    for i in range(df.shape[0]):
        for j in np.random.choice(df.shape[1],int(p*df.shape[1]),replace=False):
            should_not_swap[i,j]=1 
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Optuna

In [7]:
# Optuna for parameter search
!pip install -q optuna

import optuna
import pickle

In [8]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial, X=train[scale_features], y=train[target]):
  """
  """
  param_space = {
                    'objective': 'regression',
              'metric':'rmse',
               #'device':'gpu',  # Use GPU acceleration
               # 'gpu_platform_id': 0,
                #    'gpu_device_id': 0,
               'n_estimators':N_ESTIMATORS,
              'learning_rate':trial.suggest_uniform('learning_rate', 1e-3, 5e-2),
               'subsample': trial.suggest_uniform('subsample', 0.56, 0.76),
              'colsample_bytree':trial.suggest_uniform('colsample_bytree', 0.386, 0.586),
                'subsample_freq': trial.suggest_int('subsample_freq', 4, 6),
             #'boosting_type': 'gbdt',
               'reg_lambda':trial.suggest_uniform('reg_lambda', 34, 46),
              'reg_alpha':trial.suggest_uniform('reg_alpha', 20, 31),
            'min_child_weight':trial.suggest_uniform('min_child_weight', 400, 500),
                 'min_child_samples':trial.suggest_int('min_child_samples', 94, 118),
          'importance_type': 'gain'
             # 'num_leaves':trial.suggest_int('num_leaves', 10, 200),
              #'max_depth':trial.suggest_int('max_depth', 5, 50),
              #'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 0.005),
              #'class_weight':trial.suggest_categorical('class_weight',['balanced',None]),
              # 'n_jobs' : -1,
                #'max_bin':trial.suggest_int('max_bin', 300, 1000),
              #'cat_smooth':trial.suggest_int('cat_smooth', 5, 100),
              #'cat_l2':trial.suggest_loguniform('cat_l2', 1e-3, 100)
                }
            

  seed_list=[SEED, SEED+1]
  #kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
  #lgb_oof = np.zeros(train.shape[0])
  #lgb_pred = np.zeros(test.shape[0])
  
  X_tr, X_va, y_tr, y_va = train_test_split(X,y,test_size=.2,random_state=CVSEED)
  
  pred=np.zeros(y_va.shape[0])
  
  #for (trn_idx, val_idx) in enumerate(kf.split(X, y)):
  #print(f"===== fold {fold} =====")
  
  #X_tr = apply_noise_rn(X_tr)

  
  #start = time.time()
  for inseed in seed_list:
    param_space['random_state'] = inseed

    model = lgb.LGBMRegressor(**param_space)
    model.fit(
        X_tr, 
        y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric='rmse',
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        verbose=VERBOSE,
    )


    pred += model.predict(X_va) / len(seed_list)
    
  #elapsed = time.time() - start
  rmse = mean_squared_error(y_va, pred, squared=False)
  #print(f"fold {fold} - lgb rmse: {rmse:.6f}, elapsed time: {elapsed:.2f}sec\n")            
  
  #print(f"oof lgb_rmse = {mean_squared_error(train[target], lgb_oof, squared=False)}")

  
  return rmse

In [9]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-08-21 04:13:06,749] A new study created in memory with name: no-name-cf6f382e-3977-4728-9d84-0ac2baadb394


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 7.76623
[2000]	valid_0's rmse: 7.75951
[3000]	valid_0's rmse: 7.75829
Early stopping, best iteration is:
[3130]	valid_0's rmse: 7.75782
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 7.76699
[2000]	valid_0's rmse: 7.75942
[3000]	valid_0's rmse: 7.75654
Early stopping, best iteration is:
[3043]	valid_0's rmse: 7.75627


[I 2021-08-21 04:16:37,011] Trial 0 finished with value: 7.75486952004987 and parameters: {'learning_rate': 0.01376508999204714, 'subsample': 0.6948557461507425, 'colsample_bytree': 0.5127040908887421, 'subsample_freq': 4, 'reg_lambda': 42.010560753005905, 'reg_alpha': 27.268524196259943, 'min_child_weight': 455.1441524357472, 'min_child_samples': 103}. Best is trial 0 with value: 7.75486952004987.


Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[376]	valid_0's rmse: 7.76911
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[496]	valid_0's rmse: 7.76599


[I 2021-08-21 04:17:24,337] Trial 1 finished with value: 7.763225243805257 and parameters: {'learning_rate': 0.048983512087310374, 'subsample': 0.7381746742428232, 'colsample_bytree': 0.5458766566819864, 'subsample_freq': 4, 'reg_lambda': 39.32802748306938, 'reg_alpha': 23.62970914203234, 'min_child_weight': 432.6772664383597, 'min_child_samples': 97}. Best is trial 0 with value: 7.75486952004987.


...

Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 7.77144
[2000]	valid_0's rmse: 7.76187
[3000]	valid_0's rmse: 7.76004
Early stopping, best iteration is:
[3146]	valid_0's rmse: 7.75968
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 7.77147
[2000]	valid_0's rmse: 7.76292
Early stopping, best iteration is:
[2647]	valid_0's rmse: 7.76099


[I 2021-08-21 06:14:00,541] Trial 28 finished with value: 7.7587944834151035 and parameters: {'learning_rate': 0.011148068184459192, 'subsample': 0.6465143129883181, 'colsample_bytree': 0.4313461882442464, 'subsample_freq': 6, 'reg_lambda': 42.442262146094826, 'reg_alpha': 30.04616278862581, 'min_child_weight': 448.90310479708484, 'min_child_samples': 114}. Best is trial 0 with value: 7.75486952004987.


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 7.76442
[2000]	valid_0's rmse: 7.75796
Early stopping, best iteration is:
[2223]	valid_0's rmse: 7.7575
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 7.76693
[2000]	valid_0's rmse: 7.76056
Early stopping, best iteration is:
[2624]	valid_0's rmse: 7.75913


[I 2021-08-21 06:16:21,484] Trial 29 finished with value: 7.756282319048207 and parameters: {'learning_rate': 0.015162879758868255, 'subsample': 0.7014794041975789, 'colsample_bytree': 0.38999127317257914, 'subsample_freq': 4, 'reg_lambda': 39.37255344999815, 'reg_alpha': 23.591726418742564, 'min_child_weight': 434.5570113577248, 'min_child_samples': 103}. Best is trial 0 with value: 7.75486952004987.


Number of finished trials: 30
Best trial: {'learning_rate': 0.01376508999204714, 'subsample': 0.6948557461507425, 'colsample_bytree': 0.5127040908887421, 'subsample_freq': 4, 'reg_lambda': 42.010560753005905, 'reg_alpha': 27.268524196259943, 'min_child_weight': 455.1441524357472, 'min_child_samples': 103}


In [10]:
study.best_params

{'learning_rate': 0.01376508999204714,
 'subsample': 0.6948557461507425,
 'colsample_bytree': 0.5127040908887421,
 'subsample_freq': 4,
 'reg_lambda': 42.010560753005905,
 'reg_alpha': 27.268524196259943,
 'min_child_weight': 455.1441524357472,
 'min_child_samples': 103}

# Log

7.770644795373356 row-wise noise ver1

7.756419562622152 no noise ver5

7.755642795995508 no noise ver7 (narrow space)

7.770199059325497 random noise ver6


kfold random noise ver3
7.846914982754461